In [ ]:
import os
import operator
import time
import numpy as np
from sklearn.utils import shuffle
from sklearn.metrics import average_precision_score
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
import torch.nn.functional as F
from tqdm import tqdm
import logging
%load_ext autoreload
%autoreload 2


In [ ]:
from google.colab import drive
drive.mount("mnt")

In [ ]:
%cd "mnt/My Drive/Code"

In [ ]:
!pip3 install import_ipynb
import import_ipynb

In [ ]:
!pip install utils
import utils

In [ ]:
import net
import data_loader


In [ ]:
# Helper Functions i.e. json_loader"
import helper_functions

In [ ]:
def load_checkpoint(filename, model, optimizer=None):
    """Loads model parameters (state_dict) from a reload file. 
    Args:
        filename: (string) filename which contains the previous weights
        model: (torch.nn.Module) model to which the weights are being loaded
        optimizer: (torch.optim) the optimizer being used
    """
    if not os.path.exists(filename):
        raise("File doesn't exist {}".format(filename))
    filename = torch.load(filename)
    model.load_state_dict(filename['state_dict'])

    if optimizer:
        optimizer.load_state_dict(filename['optim_dict'])

    return filename

In [ ]:

def save_checkpoint(state, outputpath,is_best=False):
    """Saves the model as a checpoint
    Args:
        state: (dict) Models state dict
        outputpath: (string) Output Path
        is_best: (bool) Check if the model is best
    """
    filepath = os.path.join(outputpath+"model_weights.tar")
    if is_best:
      filepath=filepath+".best"
    if not os.path.exists(outputpath):
        print("Checkpoint Directory does not exist! Making directory {}".format(outputpath))
        os.mkdir(outputpath)
    else:
        print("Checkpoint Directory exists! {}",filepath)
    torch.save(state, filepath)

In [ ]:
def train(model, optimizer, loss_fn, dataloader, metrics, params, epoch):
    # set model to training mode
    model.train()
    loss = 0
    correct = 0
    test_loss=0
    # summary for current training loop and a running average object for loss
    summ = []
    nProcessed = 0
    loss_avg = helper_functions.RunningAverage()
    nTrain = len(dataloader.dataset)
    # Use tqdm for progress bar
    mAP=0
    with tqdm(total=len(dataloader)) as t:
        for i, (train_batch, labels_batch) in enumerate(dataloader):
            #train_batch = train_batch.view(train_batch.shape[0], -1)
            # move to GPU if available
            if params.cuda:
                train_batch, labels_batch = train_batch.cuda(async=True), labels_batch.cuda(async=True)

            # convert to torch Variables
            train_batch, labels_batch = Variable(train_batch), Variable(labels_batch)
            # compute model output and loss
            # clear previous gradients, compute gradients of all variables wrt loss
            optimizer.zero_grad()
            output_batch = model(train_batch)

            #loss = F.nll_loss(output_batch, labels_batch)
            loss += loss_fn(output_batch, labels_batch)

            #loss.backward()

            # performs updates using calculated gradients
            optimizer.step()
            partialEpoch = epoch + i / len(dataloader) - 1
            
            nProcessed += len(train_batch)
            pred = output_batch.data.max(1)[1] # get the index of the max log-probability
            correct += pred.eq(labels_batch.data).cpu().sum()
            if params.mAP == 1:
              mAP=net.mapk(output_batch.data,labels_batch.data)


    test_loss = loss
    test_loss /= len(dataloader) # loss function already averages over batch size
    nTotal = len(dataloader.dataset)
    acc = 100.*correct/nTotal
    print('\nTrain set: Average loss: {:.4f}, Acc: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, nTotal, acc))
    if params.mAP == 1:
      print("Metric is mAP")
      return mAP
    return acc




In [ ]:
def evaluate(model, loss_fn, dataloader, metrics, params, num_classes, epoch , eva=False):

    # set model to evaluation mode
    model.eval()

    # summary for current eval loop
    summ = []
    loss = 0
    correct = 0
    test_loss=0
    # compute metrics over the dataset
    cm = []
    for i, (data_batch, labels_batch) in enumerate(dataloader):
        # move to GPU if available
        if params.cuda:
            data_batch, labels_batch = data_batch.cuda(async=True), labels_batch.cuda(async=True)
        # fetch the next evaluation batch
        data_batch, labels_batch = Variable(data_batch), Variable(labels_batch)
        
        # compute model output
        output_batch = model(data_batch)
        loss += loss_fn(output_batch, labels_batch).item()
        #loss += F.nll_loss(output_batch, labels_batch).item()
        pred = output_batch.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(labels_batch.data).cpu().sum()

    test_loss = loss
    test_loss /= len(dataloader) # loss function already averages over batch size
    nTotal = len(dataloader.dataset)
    acc = 100.*correct/nTotal
    print('\nTest set: Average loss: {:.4f}, Acc: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, nTotal, acc))
    return acc

In [ ]:
def train_and_evaluate(model, train_dataloader, val_dataloader, optimizer, loss_fn, metrics, params, model_dir,
                       restore_file=None):


    # reload weights
    if restore_file is not None:
        restore_path = os.path.join(model_dir+restore_file)
        logging.info("Restoring parameters from {}".format(restore_path))
        load_checkpoint(restore_path, model, optimizer)
    
    scheduler = None
    if hasattr(params,'lr_decay_gamma'):
        scheduler = StepLR(optimizer, step_size=params.lr_decay_step, gamma=params.lr_decay_gamma)
    best_acc= 0.0
    map_train=[]
    map_test=[]
    for epoch in range(params.num_epochs):
        # Run one epoch
        logging.info("Epoch {}/{}".format(epoch + 1, params.num_epochs))

        if scheduler is not None:
            scheduler.step()
        print(" I have finally goten to training")
        # compute number of batches in one epoch (one full pass over the training set)
        if epoch>40:
          map_train.append(train(model, optimizer, loss_fn, train_dataloader, metrics, params, epoch))
        # Evaluate for one epoch on validation set
          map_test.append(evaluate(model, loss_fn, val_dataloader, metrics, params, 11, epoch))
        else:
          train(model, optimizer, loss_fn, train_dataloader, metrics, params, epoch)
          evaluate(model, loss_fn, val_dataloader, metrics, params, 11, epoch)


        is_best=False
        if is_best==True:
          save_checkpoint({'epoch': epoch + 1,'state_dict': model.state_dict(),'optim_dict' : optimizer.state_dict()},
                               filename=model_dir,is_best=True)
        else:
          save_checkpoint({'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optim_dict' : optimizer.state_dict()},
                               model_dir)

        # If best_eval, best_save_path
        if is_best:
            logging.info("- Found new best accuracy")
            #best_val_met = val_acc

            # Save best val metrics in a json file in the model directory
            best_json_path = os.path.join(model_dir, "metrics_val_best_weights.json")
            helper_functions.save_dict_to_json(val_metrics, best_json_path)
    return map_train,map_test



In [ ]:
def create_logger(filepath):

    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(filepath+"train.log")
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)  
        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


In [ ]:
def createNetwork(params_path,output_dir,num_classes,restore_file=None):
  #Get Path for Hyper Parameters
  json_path = os.path.join(params_path, 'params.json')
  assert os.path.isfile(json_path), "No json configuration file found at {}".format(json_path)
  params = helper_functions.Json_Parser(json_path) #Load the json file
  print(params)

  dataloaders = data_loader.fetch_dataloader(['train', 'val'], "/content/mnt/My Drive/Dataset_bk/Spectograms/", params, mixing=True)
  train_dl = dataloaders['train']
  val_dl   = dataloaders['val']
  #train_dl=torch.utils.data.DataLoader(train_dataset,batch_size=params.batch_size,shuffle=True)
  #val_dl=torch.utils.data.DataLoader(valid_dataset,batch_size=params.batch_size,shuffle=True)

  # use GPU if available
  params.cuda = torch.cuda.is_available()
  # Set the random seed for reproducible experiments
  torch.manual_seed(1240)
  if params.cuda:
    torch.cuda.manual_seed(1240)
  create_logger(output_dir)
  
  # Create the input data pipeline
  logging.info("Loading the datasets...")

  # Define the model and optimizer
  if params.model == 1:
    logging.info('  -- Training using DenseNet')

    model = net.DenseNet(growthRate=params.growthRate, depth=params.depth, reduction=params.reduction,
                            bottleneck=True, nClasses=num_classes).cuda() if params.cuda else net.DenseNet(growthRate=params.growthRate, 
                            depth=params.depth, reduction=params.reduction,
                            bottleneck=True, nClasses=num_classes)
  elif params.model == 2:
    logging.info('  -- Training using ResNet')
    model = net.ResNet(params,num_classes).cuda() if params.cuda else net.ResNet(params,num_classes)

  elif params.model == 3:
    logging.info('  -- Training using SqueezeNet')
    model = net.SqueezeNet(input_shape=(3, 128, 128), nb_classes=num_classes)

  elif params.model == 4:
    logging.info('  -- Training using Inception')
    model = net.InceptionNet(params,num_classes).cuda() if params.cuda else net.InceptionNet(params,num_classes)

  # optimizer from pytorch
  if params.optimizer == 1:
    logging.info('  ---optimizer is Adam')
    optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)
    if hasattr(params,'lambd'):
      optimizer = optim.Adam(model.parameters(), lr=params.learning_rate, weight_decay=params.lambd)
  elif params.optimizer == 2:
    logging.info('  ---optimizer is SGD')
    optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=params.learning_rate)
    if hasattr(params,'lambd'):
      optimizer = optim.SGD(model.parameters(), lr=params.learning_rate, momentum=0.9, weight_decay=params.lambd)
  loss_fn=None

  print('  ---loss function is MSE'); print('')
  loss_fn=net.loss_fn


  logging.info("Starting training for {} epoch(s)".format(params.num_epochs))
  return train_and_evaluate(model, train_dl, val_dl, optimizer, loss_fn, net.metrics, params, params_path,
                       restore_file)




In [ ]:
inc_train_001,inc_test_001=createNetwork('/content/mnt/My Drive/Code/HyperParameters/inception-v4_01/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
res_train_001,res_test_001=createNetwork('/content/mnt/My Drive/Code/HyperParameters/resnet_01/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
res_train_002,res_test_002=createNetwork('/content/mnt/My Drive/Code/HyperParameters/resnet_02/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
res_train_003,res_test_003=createNetwork('/content/mnt/My Drive/Code/HyperParameters/resnet_03/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
res_train_004,res_test_004=createNetwork('/content/mnt/My Drive/Code/HyperParameters/resnet_04/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
import  matplotlib.pyplot as plt
print(res_train_001)
print(res_train_002)
print(res_train_003)
print(res_train_004)
print(res_test_001)
print(res_test_002)
print(res_test_003)
print(res_test_004)

In [ ]:
plt.figure(figsize=(14, 5))
x=[*range(10, len(res_train_001)+10-1, 1)]
x.insert(0,0)
train_line,=plt.plot(x ,res_train_001, 'k')
train_line.set_label('Resnet- Training')
test_line,=plt.plot(x ,res_test_001, 'b')
test_line.set_label('Resnet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')

In [ ]:
plt.figure(figsize=(14, 5))
x=[*range(10, len(res_train_002)+10-1, 1)]
x.insert(0,0)
train_line,=plt.plot(x ,res_train_002, 'k')
train_line.set_label('Resnet- Training')
test_line,=plt.plot(x ,res_test_002, 'b')
test_line.set_label('Resnet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')

In [ ]:
plt.figure(figsize=(14, 5))
x=[*range(10, len(res_train_003)+10-1, 1)]
x.insert(0,0)
train_line,=plt.plot(x ,res_train_003, 'k')
train_line.set_label('Resnet- Training')
test_line,=plt.plot(x ,res_test_003, 'b')
test_line.set_label('Resnet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')

In [ ]:
plt.figure(figsize=(14, 5))
x=[*range(10, len(res_train_004)+10-1, 1)]
x.insert(0,0)
train_line,=plt.plot(x ,res_train_004, 'k')
train_line.set_label('Resnet- Training')
test_line,=plt.plot(x ,res_test_004, 'b')
test_line.set_label('Resnet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')

In [ ]:

#train_dataset=BirdDataset('/content/mnt/My Drive/Dataset_bk/traindataset.csv')
#valid_dataset=BirdDataset('/content/mnt/My Drive/Dataset_bk/valdataset.csv')
train_001,test_001=createNetwork('/content/mnt/My Drive/Code/HyperParameters/densenet01/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
train_002,test_002=createNetwork('/content/mnt/My Drive/Code/HyperParameters/densenet02/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
train_003,test_003=createNetwork('/content/mnt/My Drive/Code/HyperParameters/densenet03/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
train_004,test_004=createNetwork('/content/mnt/My Drive/Code/HyperParameters/densenet04/','/content/mnt/My Drive/Dataset_bk/Output/',11)

In [ ]:
import  matplotlib.pyplot as plt
print(train_001)
print(train_002)
print(train_003)
print(train_004)
print(test_001)
print(test_002)
print(test_003)
print(test_004)


In [ ]:
plt.figure(figsize=(14, 5))
x=[*range(10, len(train_001)+10-1, 1)]
x.insert(0,0)
train_line,=plt.plot(x ,train_001, 'k')
train_line.set_label('DenseNet- Training')
test_line,=plt.plot(x ,test_001, 'b')
test_line.set_label('DenseNet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')



In [ ]:
plt.figure(figsize=(14, 5))
x=[*range(10, len(train_002)+10-1, 1)]
x.insert(0,0)
train_line,=plt.plot(x ,train_002, 'k')
train_line.set_label('DenseNet- Training')
test_line,=plt.plot(x ,test_002, 'b')
test_line.set_label('DenseNet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')



In [ ]:
plt.figure(figsize=(14, 5))
x=[*range(10, len(train_003)+10-1, 1)]
x.insert(0,0)
train_line,=plt.plot(x ,train_003, 'k')
train_line.set_label('DenseNet- Training')
test_line,=plt.plot(x ,test_003, 'b')
test_line.set_label('DenseNet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')


In [ ]:
plt.figure(figsize=(14, 5))

x=[*range(10, len(train_004)+10-1, 1)]
x.insert(0,0)

train_line,=plt.plot(x ,train_004, 'k')
train_line.set_label('DenseNet- Training')
test_line,=plt.plot(x ,test_004, 'b')
test_line.set_label('DenseNet- Test')
axes = plt.gca()
axes.set_ylim([0,100])
plt.legend()
plt.ylabel("mAP")
plt.xlabel("No of Epochs")
plt.yscale("linear")
plt.yticks(np.arange(0, 100, 5.0))
plt.grid(True)
plt.axis('tight')